In [2]:
import re
from collections import Counter
import os
from pprint import pprint
from pymystem3 import Mystem
import tqdm
from tqdm import tqdm
import pandas as pd

### Предобработка

In [32]:
def load_stop_words(stop_filename):
	''' загрузить список стоп-слов из файла, одно слово на строке '''
	with open(stop_filename, encoding = 'utf-8') as f:
		stopwords = [w.strip() for w in f.readlines()]
	return set(stopwords)

stop_words = load_stop_words("stoplist_russian.txt")
print(stop_words)

{'со', 'по', 'не', 'на', 'что', 'каждый', 'пока', 'здесь', 'этот', 'во', 'едва', 'примечательно', 'главный', 'без', 'над', 'ещё', 'рамка', 'с', 'восемь', 'по-прежнему', 'самый', 'стать', 'год', 'я', 'четыре', 'даже', 'в', 'уже', 'она', 'первый', 'еще', 'после', 'он', 'и', 'свой', 'лишь', 'а', 'два', 'ли'}


In [33]:
m = Mystem()
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    return clean_text

def lemmatize(input):
    return [lemma.strip() for lemma in m.lemmatize(preprocessing(input.lower())) if lemma.strip()]

def remove_stop_words(lemmas, stopwords):
    return ' '.join([word for word in lemmas if word not in stopwords])

### Корпус

В качестве корпуса мы возьмем два файла, каждый из которых включает примерно 7000-1000 контекстов со словом "язык".
Каждый контекст написан на новой строке. Примеры из файла language.txt (контексты со значением "орган") получают значение "0", а из файла tongue.txt (контексты со значением как в сочетании "русский язык") значение 1.

In [214]:
language = pd.read_csv("language2.csv", header=None)
language = list(language[0].values)
len(language)

725

In [215]:
for i,sent in enumerate(language):
    language[i] = [0, remove_stop_words(lemmatize(preprocessing(language[i])), stop_words)]

In [216]:
language[:10]

[[0, 'трудно даваться математика русский рваться сюда суббота'],
 [0, 'широкий пасть задний дверь вывалиться аппарель гроб скользнуть'],
 [0, 'касание к небо игорь переключать воспроизведение'],
 [0,
  'навстречу слюнявый пасть приветственный вывалить радостно помахивать хвост'],
 [0, 'удерживаться лизнуть шершавый мордочка любимая видеть темный'],
 [0,
  'оказываться необходимый чтобы говорить публицистический который данный случай'],
 [0,
  'весь сводка опубликовывать французский отсутствовать точный координата станция баро'],
 [0, 'огромный черный весь время вываливаться изо рот'],
 [0, 'поблескивать как пуговка черный то дело вываливаться изо'],
 [0, 'италия где давать урок греческий петрарка']]

In [217]:
tongue = pd.read_csv("tongue2.csv", header=None, delimiter=';')
tongue = list(tongue[0].values)
len(tongue)

701

In [218]:
for i,sent in enumerate(tongue):
    tongue[i] = sent.lower()
    tongue[i] = re.sub(' язык.*?( |$)', ' ', sent)
    tongue[i] = [1, remove_stop_words(lemmatize(preprocessing(tongue[i])), stop_words)]

In [219]:
tongue[:10]

[[1, 'сила оставаться только копченый олений'],
 [1, 'филе пулярка устрица жареный бараний язык соус рамолад куриный'],
 [1, 'бутылка пиво корона вытрясать розовый язык светлый капля'],
 [1,
  'давать сигнал мышца ротовой полость гибкий язык передвигать подсолнух к коренной'],
 [1,
  'липкий бумажка разевать рот облизывать красный язык сладкий игольчатый холодок'],
 [1, 'движение воздух вытягивать собачий язык вызывать дыхание который'],
 [1, 'выглядывать красный глазок высовываться ржавый'],
 [1, 'мститель лаэрт фехтовальщик кончик красный язык который так легко'],
 [1,
  'медведь дарья зуб оскаливать высовывать красный язык вытягивать лапа коготь'],
 [1, 'длинный железный коготь долгий огненный язык']]

In [220]:
data = language + tongue

In [221]:
len(data)

1426

### Векторное представление корпуса и автоматическая классификация текстов

In [222]:
# математика
import numpy as np

# разные классификаторы
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

# векторизация
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# деление корпуса train/test
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Pipeline
from sklearn.pipeline import Pipeline

In [223]:
X = [item[1] for item in data]
y = [item[0] for item in data]

In [224]:
classifiers = [('SVM', LinearSVC), ('Bayes', MultinomialNB), ('Forest', RandomForestClassifier)]
vectorizers = [('CV', CountVectorizer), ('TfIdf', TfidfVectorizer)]

In [225]:
def score_classifier(clf, metric='f1', data=X):
    scores = cross_val_score(pipeline, np.asarray(data), np.asarray(y), cv=5, scoring=metric)
    score = sum(scores) / len(scores)
    return score

In [226]:
scores = []
for clf in classifiers:
    for vctr in vectorizers:
        pipeline = Pipeline([
    ('vectorizer', vctr[1]()),
    ('classifier', clf[1]()) ])
        scores.append((clf[0], vctr[0], score_classifier(pipeline)))

In [227]:
scores.sort(key=lambda tup: tup[2])
scores

[('Forest', 'TfIdf', 0.8210713982865887),
 ('Forest', 'CV', 0.8354139810412422),
 ('SVM', 'CV', 0.8757884122655021),
 ('SVM', 'TfIdf', 0.9027268507743089),
 ('Bayes', 'TfIdf', 0.9243984636476889),
 ('Bayes', 'CV', 0.9305478962864122)]

Лучшее качество f1_score дает комбинация Count Vectorizer и Наивного Байевского классификатора - 0.93.

### Уменьшение размерности

In [228]:
cv_vectorizer = CountVectorizer()
cv = cv_vectorizer.fit_transform(X)
cv = cv.toarray()
cv.shape

(1426, 3771)

In [229]:
from sklearn.decomposition import PCA
pca = PCA()
cv_pca = pca.fit_transform(cv)

In [230]:
y = np.asarray(y)

In [232]:
print(cv_pca.shape, y.shape)
print(type(cv_pca), type(y))

(1426, 1426) (1426,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [233]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cv_pca, y, test_size=0.3)

In [234]:
clf_cv = LinearSVC()
clf_cv.fit(X_train, y_train)
y_pred_cv = clf_cv.predict(X_test)
f1_score(y_test, y_pred_cv)

0.9108433734939759

Для уменьшения размерности используем метод главных компонент PCA. Количество признаков уменьшилось с 3771 до 1426.
Качество модели LinearSVC с векторным представлением Count Vectorizer увеличилось на 0.35 и стало выше 0.9.